<a href="https://colab.research.google.com/github/nnilayy/Pytorch/blob/main/Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import torch
print(torch.__version__)

2.0.1+cu118


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Setting Up Device

In [ ]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

## Importing Dataset

In [17]:
import torchvision
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor

In [30]:
train_dataset = MNIST(root='/content/data', train=True, download=True, transform=ToTensor())
test_dataset = MNIST(root='/content/data', train=False, download=True, transform=ToTensor())

In [33]:
# Extracting features and labels for Training Data
X_train=train_dataset.data
y_train=train_dataset.targets

# Extracting features and labels for Test Data
X_test=test_dataset.data
y_test=test_dataset.targets

In [39]:
X_train.shape

torch.Size([60000, 28, 28])

## Train Test Valid Split

In [34]:
def ttv_split(data,train_split,test_split,seed=None):
  if train_split+test_split>100:
    raise Exception("Train, Test Split Should not sum to more than 100%")
  train, test, val  = np.split(data.sample(frac=1,random_state=seed), [int((train_split/100)*len(data)), int(((train_split/100)+(test_split/100))*len(data))])
  return train, test, val

def train_test_valid_split(features,labels,seed,train_split,test_split):
  X_train, X_test, X_val=ttv_split(data=features,seed=seed,train_split=train_split,test_split=test_split)
  y_train, y_test, y_val=ttv_split(data=labels,seed=seed,train_split=train_split,test_split=test_split)
  return (X_train,y_train),(X_test,y_test),(X_val,y_val)

In [ ]:
(X_train,y_train),(X_test,y_test),(X_val,y_val)=train_test_valid_split(features=X,
                                                                       labels=y,
                                                                       seed=40,
                                                                       train_split=60,
                                                                       test_split=20)
X_train

## Custom Dataset and DataLoader

In [36]:
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
def MyDataset(features,labels,transform=None):
  def __init__(self):
    # self.dataset=dataset
    self.features=self.dataset.drop(-1)
    self.labels=self.dataset
    self.transform=transform

  def __len__(self):
    return len(self.annotations)

  def __getitem__(self,index):
    img_path=os.path.join(self.root_dir,self.annotations.iloc[index,0])
    y_label=torch.tensor(int(self.annotations.iloc[index,1]))
    return (image,y_label)
# train_loader=DataLoader(dataset=dataset,batch_size=16,shuffle=True,num_worker=4,pin_memory=False)

In [ ]:
train_set=MyDataset(X_train,y_train)
test_set=MyDataset(X_test,y_test)
val_set=MyDataset(X_val,y_val)

In [38]:
batch_size=64
train_loader=DataLoader(train_dataset, batch_size=batch_size,shuffle=True, num_workers=2)
test_loader=DataLoader(test_dataset, batch_size=batch_size,shuffle=False, num_workers=2)
# val_loader=DataLoader(val_set, batch_size=128,shuffle=False, num_workers=2)

## Custom Model

In [ ]:
class MyModel(nn.Module):
  def __init__(self):
    super(MyModel, self).__init__()
    self.linear1=torch.nn.Linear(100,200)
    self.activation=torch.nn.ReLU()
    self.linear2=torch.nn.Linear(200,10)
    self.softmax=torch.nn.Softmax()

    def forward(self,x):
      x=self.linear1(x)
      x=self.activation(x)
      x=self.linear2(x)
      x=self.softmax(x)
      return x

In [55]:
x,y=train_dataset[0]
x.reshape(-1,784).shape

torch.Size([1, 784])

In [ ]:
model=MyModel()
model.to(device)

In [81]:
# Model, Training and testing Loop Setup
import torch.nn.functional as F
epochs=100
learning_rate=1e-4
optimizer=torch.optim.Adam(model.parameters(),lr=learning_rate)
loss_function=nn.CrossEntropyLoss()

In [43]:
from torch import nn
class Net(nn.Module):
    ''' Models a simple Convolutional Neural Network'''
    def __init__(self):
      super(Net, self).__init__()
      self.conv1 = nn.Conv2d(3, 6, 5)
	# Max pooling over a (2, 2) window
      self.pool = nn.MaxPool2d(2, 2)
      self.conv2 = nn.Conv2d(6, 16, 5)
      self.fc1 = nn.Linear(16 * 5 * 5, 120)# 5x5 from image dimension
      self.fc2 = nn.Linear(120, 84)
      self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
      x = self.pool(F.relu(self.conv1(x)))
      x = self.pool(F.relu(self.conv2(x)))
      x = x.view(-1, 16 * 5 * 5)
      x = F.relu(self.fc1(x))
      x = F.relu(self.fc2(x))
      x = self.fc3(x)
      return x

net = Net()
print(net)


Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


## Training, Testing and Fit Loops and Functions

In [26]:
# Training Function and Loop
def train(train_loader,model,loss_function,optimizer):
  model.train()
  train_loss=0

  for features, labels in train_loader:
    features, labels = features.to(device), labels.to(device)
    predictions=model(features)
    loss=loss_function(predictions,labels)
    train_loss+=loss.item()

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  num_batches=len(train_loader)
  train_loss_per_epoch=train_loss/num_batches
  print("Train Loss:",train_loss_per_epoch)

In [25]:
# Testing Function and Loop
def test(test_loader,model,loss_function):
  model.eval()
  test_loss=0

  with torch.no_grad():
    for features, labels in test_loader:
      features, labels = features.to(device), labels.to(device)
      predictions=model(features)
      loss=loss_function(predictions,labels)
      test_loss+=loss.item()

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

    num_batches=len(test_loader)
    test_loss_per_epoch=test_loss/num_batches
    print("Test Loss:",test_loss_per_epoch)

In [ ]:
# Fit Function
for epoch in range(100):
  print("epoch: ",epoch)
  train(train_loader,model,loss_function,optimizer)
  test(test_loader,model,loss_function)

## Tensorflow Like Style for Training Loss and Accuracy

In [ ]:
print(f"Epoch {epoch+1}/{num_epochs} - Batch {batch_idx+1}/{len(dataloader)} - Loss: {loss.item()}\r", end='')

In [51]:
import time
for i in range(10):
  print(f"{i}",end='')
  time.sleep(0.7)
  print("\r",end='')

## Saving Model

In [ ]:
torch.save(model.state_dict(),'model.pt')

## Loading Model

# DUMPSTER


## Imp Random

In [ ]:
    # def forward(self,x):
    #   xb=xb.reshape(-1,784)
    #   out=self.linear(xb)
    #   return out

    # def training_step(self,batch):
    #   images,labels=batch
    #   outputs=self(images)
    #   loss=F.cross_entropy(outputs,labels)
    #   return loss

    # def validation_step(self,batch):
    #   images, labels=batch
    #   out=self(images)
    #   loss=F.cross_entropy(outputs,labels)
    #   acc=accuracy(outputs,labels)
    #   return {'val_loss':loss,'val_acc':acc}


# for parameter in model.parameters():
# print(parameter.shape
# model.linear1
# model.linear2
# model.activation
# model.softmax


In [ ]:
 model.linear1

In [ ]:
x=[1,2,3,4,5,6,7,8,9,10]
x_square=[sample**2 for sample in x]
x_square

In [ ]:
# Validation
def evaluate(model,val_loader):
  outputs=[models.validation_step(batch) for batch in val_loader]
  return model.validation_epoch_end(outputs)

In [ ]:
# Training, Val, Test Loop
for epoch in range(100):
  for features, labels in train_loader:
    features, labels = features.to(device), labels.to(device)
    predictions=model(features)
    loss=loss_function(predictions,labels)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

## Best Practices

In [ ]:
# https://www.youtube.com/watch?v=O2wJ3tkc-TU
# https://www.youtube.com/watch?v=2AhiHV7QGVk

## Transfer Learning and HyperParameter Training

In [ ]:
# https://www.youtube.com/watch?v=qaDe0qQZ5AQ

In [ ]:
torch.min(image).item()

## Custom Activation Function, Loss Function, Optimzer:

In [ ]:
# Custom Optimizer:
# https://www.youtube.com/watch?v=zvp8K4iX2Cs&pp=ygUUY3VzdG9tIG1vZGVsIHB5dG9yY2g%3D

In [ ]:
# https://www.youtube.com/watch?v=SDPeeX6LEnk []
# https://www.youtube.com/watch?v=OIenNRt2bjg []
# https://www.youtube.com/watch?v=H69j69FFMV0 [Dataset Spli First, then Data Loader]
# https://www.youtube.com/watch?v=SDPeeX6LEnk

In [ ]:
from torch import nn

class MyModel(nn.Module):
  def __init__(self): #Initializing the MyModel Class
    super().__init__() #Calling The Parent Class


## Other Dumpster

In [ ]:
data=torch.tensor([1,2,3,4,5])
print(data)
print(data.shape)

In [ ]:
#Tensor Gradients
x=torch.tensor(3.)
w=torch.tensor(4.,requires_grad=True)
b=torch.tensor(5.,requires_grad=True)

In [ ]:
# Calculate the value of y
y=x*w+b
y

In [ ]:
data=torch.full((3,3),101)
data

In [ ]:
#Converts numpy array to tensor
import numpy as np
x=np.ones((3,2))
y=torch.from_numpy(x)
x.dtype, y.dtype
# Internally both are the same

In [ ]:
# Convert Tensor to numpy
x=torch.ones((3,2))
y=x.numpy()
x.dtype, y.dtype

In [ ]:
# Find the number of element in a tensor
x.numel()

In [ ]:
# if gradient +ve:
#   increasing the weight value increases loss
#   decreasing the weight value decreases loss

# if gradient -ve:
#   increasing the weight value decreases loss
#   decreasing the weight value increases loss

In [ ]:
value=value-stepsize
value=value-lr*slope
value=value-lr*gradient
value=value-lr*derivative

In [ ]:
import torch.nn.functional as F
loss_func=F.mse_loss
loss=loss_func(model(input),labels)

In [ ]:
# optimizers perform the gradient descent, that is they adjust the weights and biases of parameters
optim=torch.optim.SGD(model.parameters,lr=1e-5)

In [ ]:
# def fit(num_epochs,model,loss_func,optim,train):
for epoch in range(num_epochs):
  for x,y in train:
    prediction=model(x)
    loss=loss_func(prediction,y)
    loss.backward()

    # Optimizer performs gradient descent and updates the Weights and Biases of Features/Parameters
    optim.step()

    # Reset the gradients for next epoch
    optim.zero_grad()

In [ ]:
import torch
import torchvision
from torchvision.datasets import MNIST
import torchvision.transforms as transforms

In [ ]:
# Transforms are used for handling image data one such usage is ToTensor() function

In [ ]:
# Random split in Pytorch
from torch.utils.data import random_split
train_ds, val_ds - random_split(dataset, [50000, 10000])
len (train_ds), len(val_ds)

In [ ]:
# Dataloader
from torch.utils.data import DataLoader
batch_size=128
train_loader=DataLoader(train_data,batch_size,shuffle=True) #Shuffle=True so that with each epoch the batches are not send in the same order, this helps to generalize model better
val_loader=DataLoader(val_data,batch_size)

In [ ]:
# Models
images.reshape(128,784)
model.parameters()
model.linear.weights.shape
model.linear.bias.shape

In [ ]:
import pandas as pd
import numpy as np
numbers = np.arange(1, 21)
labels = np.random.randint(0, 2, size=20)
df = pd.DataFrame({'Numbers': numbers, 'Labels': labels})
df

In [ ]:
X=df.iloc[:,:-1]
y=df.iloc[:,-1:]